In [1]:
def parse_input(filename):
    with open(filename, 'r') as f:
        # Split the input into two parts based on double newlines
        parts = f.read().strip().split('\n\n')
        
        # Parse the grid
        grid = [list(line) for line in parts[0].split('\n')]
        
        # Parse the instructions - remove all newlines and keep as one string
        instructions = parts[1].replace('\n', '')
        
        return grid, instructions



In [2]:
def find_start_position(grid):
    for y, row in enumerate(grid):
        for x, cell in enumerate(row):
            # print(cell)
            if cell == '@':
                return (x, y)
    return None

def move(pos, instruction):
    # print("starting move", pos, instruction)
    x, y = pos
    if grid[y][x] == '#':
        return False
    if grid[y][x] == '.':
        return True

    if instruction == '^':
        direction= (0, - 1)
    elif      instruction == 'v':

        direction= (0,  1)
    elif      instruction == '>':

        direction= ( 1, 0)
    elif      instruction == '<':

        direction = ( - 1, 0)
    else:
        raise ValueError(f"Invalid direction: {instruction}")
    if move((x + direction[0], y + direction[1]), instruction) == True:
        # print("Moving!!!")
        currentChar = grid[y][x]
        grid[y][x] = grid[y + direction[1]][x + direction[0]]
        grid[y + direction[1]][x + direction[0]] = currentChar
        return True
def score(grid):
    scores = []
    for y, row in enumerate(grid):
        for x, cell in enumerate(row):
            if cell == 'O' or cell == '[':
                scores.append(x + 100 * y)         
    return sum(scores)

In [3]:

grid, instructions = parse_input('15.test.txt')

for i in range(len(instructions)):
    print(".", end='')
    move(find_start_position(grid), instructions[i])




# Print the grid
print("Grid:")
for row in grid:
    print(''.join(row))

print("Score:", score(grid))



............................................................................................................................................................................................................................................................................................................................................................................................................................................................................................................................................................................................................................................................................................................................Grid:
##########
#.O.O.OOO#
#........#
#OO......#
#OO@.....#
#O#.....O#
#O.....OO#
#O.....OO#
#OO....OO#
##########
Score: 10092


In [4]:
def double_grid(grid):
    newrows = []
    for i in range(len(grid)):
        newrow = []
        for j in range(len(grid[0])):
            cell = grid[i][j]
            if cell == '#':
                newrow.extend(['#', '#'])
            elif cell == '.':
                newrow.extend(['.','.'])
            elif cell == 'O':
                newrow.extend(['[',']'])
            elif cell == '@':    
                newrow.extend(['@','.'])
        newrows.append((newrow))
    return newrows


grid, instructions = parse_input('15.test.txt')
grid = double_grid(grid)

for row in grid:
    print(''.join(row))

####################
##....[]....[]..[]##
##............[]..##
##..[][]....[]..[]##
##....[]@.....[]..##
##[]##....[]......##
##[]....[]....[]..##
##..[][]..[]..[][]##
##........[]......##
####################


In [5]:
def move2(pos, instruction, test_only = False):
    # print("starting move", pos, instruction)
    x, y = pos
    if grid[y][x] == '#':
        return False
    if grid[y][x] == '.':
        return True
    is_y = False
    if instruction == '^':
        direction= (0, - 1)
        is_y = True
    elif      instruction == 'v':
        direction= (0,  1)
        is_y = True
    elif      instruction == '>':

        direction= ( 1, 0)
    elif      instruction == '<':

        direction = ( - 1, 0)
    else:
        raise ValueError(f"Invalid direction: {instruction}")
    currentChar = grid[y][x]
    if is_y == True:
        test = True
        if currentChar == '[':
            test = move2((x + 1, 0), instruction, True)
        elif currentChar == ']':
            test = move2((x - 1, 0), instruction, True)
        if test == False:
            print("Returning false because this is not a valid move in an adjacent cell")
            return False
        print("I can move the adjacent thing in this direction", )
    if move2((x + direction[0], y + direction[1]), instruction, test_only) == True:
        if test_only == False:
            if is_y == True:
                if currentChar == '[':
                    move2((x - 1, y+direction[1]), instruction, test_only)
                    print("moving adjacent thing - left", instruction, test_only, currentChar)
                elif currentChar == ']':
                    move2((x + 1, y+direction[1]), instruction, test_only)
                    print("moving adjacent thing - right")
            # print("Moving!!!")
            grid[y][x] = grid[y + direction[1]][x + direction[0]]
            grid[y + direction[1]][x + direction[0]] = currentChar
        return True

In [6]:
from IPython.display import clear_output
from time import sleep

grid, instructions = parse_input('15.test.txt')
grid = double_grid(grid)
print("Starting positions")
for row in grid:
    print(''.join(row))

# for i in range(len(grid)):
#     print(''.join(grid[i]))

print(find_start_position(grid))

for i in range(25):
    print("Move", i)
    print(instructions[i], end='')
    print('')
    move2(find_start_position(grid), instructions[i])
    # sleep(1)
    # clear_output(wait=True)
    for i in range(len(grid)):
        print(''.join(grid[i]))



Starting positions
####################
##....[]....[]..[]##
##............[]..##
##..[][]....[]..[]##
##....[]@.....[]..##
##[]##....[]......##
##[]....[]....[]..##
##..[][]..[]..[][]##
##........[]......##
####################
(8, 4)
Move 0
<
####################
##....[]....[]..[]##
##............[]..##
##..[][]....[]..[]##
##...[]@......[]..##
##[]##....[]......##
##[]....[]....[]..##
##..[][]..[]..[][]##
##........[]......##
####################
Move 1
v
I can move the adjacent thing in this direction
####################
##....[]....[]..[]##
##............[]..##
##..[][]....[]..[]##
##...[].......[]..##
##[]##.@..[]......##
##[]....[]....[]..##
##..[][]..[]..[][]##
##........[]......##
####################
Move 2
v
I can move the adjacent thing in this direction
####################
##....[]....[]..[]##
##............[]..##
##..[][]....[]..[]##
##...[].......[]..##
##[]##....[]......##
##[]...@[]....[]..##
##..[][]..[]..[][]##
##........[]......##
####################
Move 3
>
##

In [7]:
 for i in range(len(grid)):
    print(''.join(grid[i]))

####################
##....[]....[]..[]##
##............[]..##
##..[][]....[]..[]##
##...[].......[]..##
##[]##....[]......##
##[]......[]..[]..##
##..[][]...[].[][]##
##.......@[]......##
####################
